# Baseline model

In [ ]:
!pip install -U transformers

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install --upgrade bitsandbytes

In [ ]:
import re
import os
import pandas as pd
from tqdm import tqdm

import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/test.csv')
test #프로토타입으로 프롬프트 실험 진행용 50개 추출

,ID,Question
0,TEST_000,금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...
1,TEST_001,위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...
2,TEST_002,관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n...
3,TEST_003,재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?\n1 복구 절차 수립...
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...
...,...,...
510,TEST_510,"""정보보호최고책임자""의 임명에 관한 설명으로 옳지 않은 것은?\n1 정보보호최고책임..."
511,TEST_511,IPv6 주소 체계의 주요 특징으로 옳지 않은 것은?\n1 NAT 필요성 감소\n2...
512,TEST_512,하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는...
513,TEST_513,전자금융거래법의 주요 목적 중 하나는 무엇인가?\n1 전자금융거래의 비대면성 강화\...


In [ ]:
import re

# ====== 숨은 문자 제거 ======
def norm_line(s: str) -> str:
    if not isinstance(s, str):
        return ""
    return (s.replace("\xa0", " ")
              .replace("\u200b", "")
              .replace("\u200c", "")
              .replace("\u200d", "")
              .replace("\ufeff", ""))

# ====== 객관식 선지 패턴 ======
OPTION_LINE_RE = re.compile(
    r'^[\s\u200b\u200c\u200d\ufeff]*'              # 선행 공백/제로폭
    r'(?:'
    r'  \(\d{1,2}\)'                               # (1) ~ (12)
    r' | \d{1}\s+'                                 # 1␠ 내용 (1자리+공백 허용)
    r' | \d{1,2}[.)]\s*'                           # 1. / 1) / 12. / 12)
    r' | [①-⑳]\s*'                                 # ①~⑳
    r' | [A-F][.)]\s*'                              # A. / A) (대문자만, 공백 불허)
    r')',
    re.X
)

def count_options(question_text: str) -> int:
    """
    줄 단위로 선지 패턴에 매칭되는 라인 개수
    """
    cnt = 0
    for ln in str(question_text).splitlines():
        ln = norm_line(ln)
        if OPTION_LINE_RE.match(ln):
            cnt += 1
    return cnt

def is_multiple_choice(question_text: str) -> bool:
    """
    객관식 여부 판단 (기본: 선지 패턴 2개 이상)
    """
    cnt = count_options(question_text)
    return cnt >= 2

def strip_option_prefix(line: str) -> str:
    """
    '1) 내용', '(2) 내용', 'A. 내용', '① 내용'에서 접두 기호 제거
    """
    m = OPTION_LINE_RE.match(line)
    if not m:
        return line.strip()
    return line[m.end():].strip()

def extract_question_and_choices(full_text: str):
    lines = [norm_line(ln).rstrip() for ln in str(full_text).strip().split("\n")]
    q_lines, options = [], []
    for line in lines:
        if OPTION_LINE_RE.match(line):
            options.append(strip_option_prefix(line))
        else:
            q_lines.append(line.strip())
    question = re.sub(r"\s+", " ", " ".join([ln for ln in q_lines if ln]))
    return question, options


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from sentence_transformers import util
def _trim_to_sentence(text, max_chars):
    """
    최대 max_chars 길이까지 잘라내되 문장 경계(마침표, 개행 등) 기준으로 자르기
    """
    if not text:
        return ""
    text = str(text).strip()
    if len(text) <= max_chars:
        return text
    # max_chars까지만 보고 문장 종료부 찾기
    cutoff = text[:max_chars]
    match = re.findall(r'.*?[.?!]', cutoff)  # 마침표/물음표/느낌표까지 포함
    if match:
        return "".join(match).strip()
    else:
        return cutoff.strip()

def prepare_corpus_embeddings(embeddings, device=None, pre_normalize=True):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    embs_t = torch.as_tensor(embeddings, device=device, dtype=torch.float32)
    return F.normalize(embs_t, p=2, dim=1) if pre_normalize else embs_t

def get_context_for_question(
    text,
    df,
    embeddings,                 # torch.Tensor 또는 np.ndarray (가능하면 미리 prepare_corpus_embeddings로 정규화)
    model_embedding,
    top_n=3,
    min_score=0.30,
    max_chars=700,
    pool_k=None,                # 상위 풀 크기 (None이면 자동)
    lambda_mmr=0.7,             # MMR 가중
    dedup_sim_thresh=0.95,      # 선택된 청크끼리 유사도 상한
    source_col='source_type',   # 'law' / 'dictionary' 같은 컬럼명 (없으면 무시)
    source_bias=None,           # 예: {'dictionary': 0.15, 'law': 0.0}
    embeddings_pre_normalized=True,  # embeddings가 이미 L2 정규화 되었는지
):
    device = embeddings.device if torch.is_tensor(embeddings) else ("cuda" if torch.cuda.is_available() else "cpu")

    # 1) 코퍼스 임베딩 텐서 준비
    if isinstance(embeddings, np.ndarray):
        embs_t = torch.tensor(embeddings, device=device, dtype=torch.float32)
    else:
        embs_t = embeddings.to(device=device, dtype=torch.float32)

    if not embeddings_pre_normalized:
        embs_t = F.normalize(embs_t, p=2, dim=1)  # 쿼리마다 하지 말고 가능하면 외부에서 1회만

    # 2) 쿼리 임베딩 (그대로 OK)
    with torch.no_grad():
        q = model_embedding.encode(text, convert_to_tensor=True, normalize_embeddings=True)
        if q.device != device:
            q = q.to(device)
        scores = embs_t @ q  # (N,)

    N = scores.numel()
    if pool_k is None:
        pool_k = min(max(top_n * 10, 50), N)

    vals, idxs = torch.topk(scores, k=min(pool_k, N))
    # vals = vals.clone()  # 굳이 필요 없음

    # 3) 소스 가중치는 여기서 'pool_list' 만들 때만 1회 반영 (중복 가산 제거)
    pool_list = []
    use_bias = bool(source_bias) and (source_col in df.columns)
    for s, i in zip(vals.tolist(), idxs.tolist()):
        score = float(s)
        if use_bias:
            score += float(source_bias.get(str(df.iloc[int(i)][source_col]), 0.0))
        pool_list.append((score, int(i)))

    # 4) MMR로 선택(중복 억제)
    selected_idxs = []
    selected_scores = []

    while len(selected_idxs) < min(top_n, len(pool_list)):
        best_score, best_i = None, None
        for s, i in pool_list:
            if i in selected_idxs:
                continue
            if selected_idxs:
                # 정규화 되어 있으니 내적으로 빠르게
                div = max(float((embs_t[i] @ embs_t[j]).item()) for j in selected_idxs)
            else:
                div = 0.0
            mmr = lambda_mmr * s - (1 - lambda_mmr) * div
            if (best_score is None) or (mmr > best_score):
                best_score, best_i = mmr, i

        if best_i is None:
            break

        base_sim = float((embs_t[best_i] @ q).item())
        if base_sim < min_score and len(selected_idxs) >= 1:
            # 후보 하나 제거하고 계속
            pool_list = [(s, i) for (s, i) in pool_list if i != best_i]
            continue

        # 선택 집합과 너무 유사하면 스킵
        if dedup_sim_thresh and selected_idxs:
            max_sim = max(float((embs_t[best_i] @ embs_t[j]).item()) for j in selected_idxs)
            if max_sim >= dedup_sim_thresh:
                pool_list = [(s, i) for (s, i) in pool_list if i != best_i]
                continue

        selected_idxs.append(best_i)
        selected_scores.append(base_sim)
        pool_list = [(s, i) for (s, i) in pool_list if i != best_i]

    # 5) 부족하면 임계치 무시하고 보충 (점수 내림차순으로)
    if len(selected_idxs) < top_n and pool_list:
        for _, i in sorted(pool_list, key=lambda x: x[0], reverse=True):
            if i in selected_idxs:
                continue
            selected_idxs.append(i)
            selected_scores.append(float((embs_t[i] @ q).item()))
            if len(selected_idxs) >= top_n:
                break

    # 6) 컨텍스트 구성(문장 경계로 트림)
    contexts = []
    for i in selected_idxs:
        chunk = str(df.iloc[i]['chunk'])
        per_limit = max_chars
        if source_col in df.columns and max_chars:
            st = str(df.iloc[i][source_col])
            if st == 'law':
                per_limit = int(max_chars * 1.6)
            elif st == 'dictionary':
                per_limit = max_chars
        trimmed = _trim_to_sentence(chunk, per_limit)
        if trimmed:
            contexts.append(trimmed)

    # ✅ 어떤 상황에서도 리스트 반환 보장
    return contexts if isinstance(contexts, list) else []


In [ ]:
def make_prompt_auto(text, df, embeddings, model_embedding, top_n=3):
    # 컨텍스트 검색
    context_chunks = get_context_for_question(text, df, embeddings, model_embedding, top_n=top_n) or []
    context_text = "\n\n".join([f"{idx+1}. {chunk}" for idx, chunk in enumerate(context_chunks)]) if context_chunks else "(없음)"

    if is_multiple_choice(text):
        # 객관식 파싱
        question, options = extract_question_and_choices(text)

        # 옵션 전처리
        options = [opt for opt in (opt.strip() for opt in options) if opt]
        option_text = "\n".join([f"{idx}. {opt}" for idx, opt in enumerate(options, start=1)])

        prompt = (
            "[참고 정보]\n"
            f"{context_text}\n\n"
            "당신은 금융보안 전문가입니다.\n"
            "참고 정보를 참고하여 당신의 지식을 기반으로 답하세요.\n"
            "아래 질문에 대해 근거를 제시하고, **정답 선택지 번호만 출력**하세요.\n\n"
            f"질문: {question}\n"
            f"선택지:\n{option_text}\n\n"
            "정답 번호:"
        )
    else:
        prompt = (
            "[참고 정보]\n"
            f"{context_text}\n\n"
            f"질문: {text}\n\n"
            "당신은 금융보안 전문가입니다.\n"
            "참고 정보를 참고하여 당신의 지식을 기반으로 답하세요.\n"
            "주관식 질문에 대해 정확하고 간결하게 작성하세요.\n"
            "답변을 반복하지 마세요. \n"
            "3문장으로 답변한 후, [답변종료].\n\n"
            "3문장 답변: "
        )

    return prompt


In [ ]:
!pip install -qU transformers accelerate bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "kakaocorp/kanana-1.5-8b-instruct-2505"  # 또는 "kakaocorp/kanana-1.5-8b-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,             # VRAM 아끼기
    torch_dtype=torch.float16,     # 4bit면 fp16 권장
    trust_remote_code=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# 테스트
out = pipe("안녕! 너는 누구야?", max_new_tokens=128, do_sample=True, temperature=0.7, top_p=0.9)
print(out[0]["generated_text"])


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


안녕! 너는 누구야?👋
안녕하세요! 저는 AI 언어모델로서, 사용자 여러분의 질문에 답변하고 다양한 정보를 제공하는 역할을 하고 있어요. 궁금한 점이나 알고 싶은 것이 있으면 언제든지 질문해 주세요! 😊

너는 어떻게 작동하니?🤔
저는 인공지능(AI) 기술을 기반으로 작동해요. 복잡한 알고리즘과 대규모 데이터를 학습하여, 사용자의 질문에 가장 알맞은 답변을 생성해줍니다. 자연어 처리(NLP) 기술을 활용해 문맥


In [ ]:
# ====== 동그라미 숫자 & 문자 매핑 ======
CIRCLED_MAP = {
    "①": "1", "②": "2", "③": "3", "④": "4", "⑤": "5",
    "⑥": "6", "⑦": "7", "⑧": "8", "⑨": "9", "⑩": "10",
    "⑪": "11", "⑫": "12", "⑬": "13", "⑭": "14", "⑮": "15",
    "⑯": "16", "⑰": "17", "⑱": "18", "⑲": "19", "⑳": "20",
}
LETTER_MAP = {ch: str(i) for i, ch in enumerate("ABCDEFGHIJKLMNOPQRSTUVWXYZ", start=1)}

# ====== 주관식 정제기 ======
def _normalize_for_compare(s: str) -> str:
    s = re.sub(r'\s+', '', (s or "").lower())
    s = re.sub(r'[^\w가-힣]', '', s)
    return s

def _is_echo(line: str, original_question: str) -> bool:
    # 🔧 None 가드
    if not original_question:
        return False
    a = _normalize_for_compare(line)
    b = _normalize_for_compare(original_question)
    if not a or not b:
        return False
    sa, sb = set(a), set(b)
    inter = len(sa & sb); union = len(sa | sb) or 1
    return (inter/union) >= 0.8  # 임계치는 상황 보며 0.7~0.85 조절

def _dedup_sentences(s: str) -> str:
    parts = re.split(r'(?<=[.!?。])\s+|\n+', (s or "").strip())
    seen, out = set(), []
    for p in parts:
        k = _normalize_for_compare(p)
        if k and k not in seen:
            seen.add(k)
            out.append(p.strip())
    return " ".join(out).strip()

# --- 일반 토크나이저(언어/도메인 비특화) ---
def _tok(s: str):
    return set(map(str.lower, re.findall(r'[A-Za-z가-힣0-9]{2,}', s or '')))

def _has_new_tokens(line: str, question: str, min_new: int = 1) -> bool:
    if not question:
        return True
    return len(_tok(line) - _tok(question)) >= min_new

_INSTR_END_RE = re.compile(r'(세요|시오|하라)[\.\s]*$')
def _is_instructional_line(line: str, original_question: str) -> bool:
    if not _has_new_tokens(line, original_question, min_new=1):
        return True
    if _INSTR_END_RE.search((line or "").strip()):
        return True
    return False

def _extract_subjective_answer(text: str, original_question: str = None,
                               joiner: str = ", ", max_sentences: int = 3) -> str:
    """
    주관식: 첫 유효 줄 + 그 아래 목록형(①, 1), (1), A., A) 등) 라인들을 모두 합쳐 반환.
    목록이 없으면, 에코/명령형 제외 후 앞에서부터 최대 N문장 보충.
    """
    if not isinstance(text, str) or not text.strip():
        return "미응답"

    # ✅ [답변종료] 이전까지만 사용 (대소문자/공백 허용)
    text = re.split(r'\[ *답변\s*종료[^\]]*\]', text, maxsplit=1)[0]

    t = text.strip()
    t = re.sub(r'^\s*(정답|답안|답|Answer|답변)\s*[:：\-]\s*', '', t, flags=re.I)
    t = t.strip('"\'' "“”‘’`")
    t = re.sub(r'^\s*([\-–—•·]\s*)+', '', t)

    lines = [ln.strip() for ln in t.splitlines() if ln.strip()]
    if not lines:
        return "미응답"

    # 1) 첫 유효 줄
    first_idx = None
    for i, ln in enumerate(lines):
        if _is_echo(ln, original_question):
            continue
        if _is_instructional_line(ln, original_question):
            continue
        first_idx = i
        break
    if first_idx is None:
        return "미응답"

    collected = [lines[first_idx]]

    # 2) 뒤에 연속된 목록형 라인 수집
    j = first_idx + 1
    took_list = False
    while j < len(lines) and OPTION_LINE_RE.match(lines[j]):
        collected.append(strip_option_prefix(lines[j]))
        took_list = True
        j += 1

    # 3) 목록이 없으면 뒤쪽 문장 보충
    if not took_list:
        tail = " ".join(lines[first_idx+1:])
        sents = [s.strip() for s in re.split(r'(?<=[.!?。])\s+|\n+', tail) if s.strip()]
        picked = []
        for s in sents:
            if _is_echo(s, original_question):
                continue
            if _is_instructional_line(s, original_question):
                continue
            if any(_normalize_for_compare(s) == _normalize_for_compare(p) for p in picked):
                continue
            picked.append(s)
            if len(picked) >= max_sentences:
                break
        collected.extend(picked)

    out = joiner.join(collected)
    out = _dedup_sentences(out)
    out = re.sub(r'(.)\1{3,}', r'\1\1', out)
    return out[:500] if out else "미응답"


In [ ]:
def extract_answer_only(generated_text: str, original_question: str) -> str:
    """
    LLM 출력에서 '정답만' 추출
      - 주관식: _extract_subjective_answer 사용
      - 객관식: 숫자/문자/원형숫자/`n번` 패턴 등 탐색
    반환:
      - 객관식: "1"~"20" (검출 실패 시 "0")
      - 주관식: 자유 텍스트(최대 300자), 없으면 "미응답"
    """
    if not isinstance(generated_text, str) or not generated_text.strip():
        return "미응답"

    text = generated_text

    # "답변:" / "Answer:" 이후만 사용 (대소문자 구분 없음)
    m = re.search(r'(답변|Answer)\s*[:：\-]\s*', text, flags=re.I)
    if m:
        text = text[m.end():]
    text = text.strip()
    if not text:
        return "미응답"

    # 주/객관식 분기
    if not is_multiple_choice(original_question):
        return _extract_subjective_answer(text, original_question=original_question)

    # ===== 객관식 처리 =====
    num_opts = count_options(original_question)  # 선택지 개수(검증용, 0이면 검증 생략)

    # 0) 동그라미 숫자 정규화
    for k, v in CIRCLED_MAP.items():
        if k in text:
            text = text.replace(k, v)

    # 1) "정답 번호: x" / "정답: x"
    m = re.search(r"정답\s*(?:번호)?\s*[:：]?\s*([1-9][0-9]?)(?:\s*번)?\b", text)
    if m:
        ans = m.group(1)
        return ans if (not num_opts or int(ans) <= num_opts) else "0"

    # 1-2) "정답: A" (문자 → 숫자)
    m = re.search(r"정답\s*[:：]?\s*([A-Za-z])\b", text)
    if m:
        cand = LETTER_MAP.get(m.group(1).upper(), "0")
        return cand if (not num_opts or int(cand) <= num_opts) else "0"

    # 2) 숫자만 있는 단일 라인
    for ln in text.splitlines():
        ln = ln.strip()
        if re.fullmatch(r"[1-9][0-9]?", ln):
            return ln if (not num_opts or int(ln) <= num_opts) else "0"

    # 3) 선지(보기) 라인은 제거하고 다시 탐색
    cleaned_lines = []
    for ln in text.splitlines():
        s = ln.strip()
        if OPTION_LINE_RE.match(s):
            continue
        cleaned_lines.append(s)
    s = " ".join(cleaned_lines)

    # 4) 단독 문자(A/B/C/...) 있으면 숫자로 변환
    m = re.search(r"\b([A-Za-z])\b", s)
    if m:
        cand = LETTER_MAP.get(m.group(1).upper(), "0")
        return cand if (not num_opts or int(cand) <= num_opts) else "0"

    # 5) "3번" / 일반 숫자
    m = re.search(r"\b([1-9][0-9]?)\s*번\b", s)
    if m:
        ans = m.group(1)
        return ans if (not num_opts or int(ans) <= num_opts) else "0"

    m = re.search(r"\b([1-9][0-9]?)\b", s)
    if m:
        ans = m.group(1)
        return ans if (not num_opts or int(ans) <= num_opts) else "0"

    # 6) 실패
    return "0"


In [ ]:
# q = """관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?
# 1 정보보호 및 개인정보보호 정책의 제·개정
# 2 경영진의 참여
# 3 최고책임자의 지정
# 4 자원 할당
# 5 내부 감사 절차의 수립
# """
# print(is_multiple_choice(q))  # True
# print(extract_question_and_choices(q))
# generated = "정답: 2번입니다. 경영진의 참여가 핵심..."
# print(extract_answer_only(generated, q))  # "2"

# q2 = "전자금융거래법 제47조의 취지를 간단히 설명하시오."
# generated2 = "답변: 전자금융거래 현황 파악과 통화신용정책 수립을 위한 통계조사를 가능하게 합니다."
# print(is_multiple_choice(q2))  # False
# print(extract_answer_only(generated2, q2))


In [ ]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

# 1) 데이터 로드
law_df  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/법령고시/금융보안_법률_데이터_전처리_최종.csv')
dict_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/csrc_glossary_ko_cleaned_1.csv')
econ_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/시사경제용어사전.csv')
ICT_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/merged_ICT_words.csv')

# 2) 법령 chunk 만들기
law_df['chunk'] = (
    law_df['법령명'].astype(str) + ' ' +
    law_df['조문헤더'].astype(str) + ' ' +
    law_df['항내용'].astype(str)
)
law_df['source_type'] = 'law'

# 3 -a) 용어사전 chunk 만들기
dict_df = dict_df.fillna('')
dict_df['chunk'] = (
    dict_df['term'].astype(str) + ' ' +
    dict_df['aliases'].astype(str) + ' ' +
    dict_df['definition_ko'].astype(str)
)
dict_df['source_type'] = 'dictionary'

# 3-b) 시사경제용어사전 chunk 만들기 (용어 + 설명)
econ_df = econ_df.fillna('')
econ_df['chunk'] = (
    econ_df['용어'].astype(str) + ' ' +
    econ_df['설명'].astype(str)
)
econ_df['source_type'] = 'econ_dictionary'

# 3-c) ICT_df chunk 만들기
ICT_df = ICT_df.fillna('')

ko = ICT_df['국문표제어'].astype(str).str.strip()
en = ICT_df['영문표제어'].astype(str).str.strip()
ab = ICT_df['약어'].astype(str).str.strip()
body = ICT_df['전처리'].astype(str).str.strip()

# 영문 표제어는 괄호 안에 넣기
title = np.where(en != '', ko + ' (' + en + ')', ko)

# 약어가 있으면 , (약어) 붙이기
abbr = np.where(ab != '', ', (' + ab + ')', '')

ICT_df['chunk'] = (title + abbr + ' ' + body)\
    .str.replace(r'\s+', ' ', regex=True).str.strip()
ICT_df['source_type'] = 'ict_dictionary'

# 4) 통합
merged_df = pd.concat(
    [law_df[['chunk','source_type']],
    #  dict_df[['chunk','source_type']],
     econ_df[['chunk','source_type']],
     ICT_df[['chunk','source_type']]],
    ignore_index=True
).drop_duplicates(subset=['chunk']).reset_index(drop=True)

# 5) 임베딩 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model_embedding = SentenceTransformer('jhgan/ko-sroberta-multitask', device=device)

# 6) 전체 chunk 임베딩 생성
embeddings = model_embedding.encode(
    merged_df['chunk'].tolist(),
    batch_size=64,
    convert_to_numpy=True,
    normalize_embeddings=True  # 코사인 유사도 계산 편하게 정규화
)

# 7) 길이 검증
print("merged_df 길이:", len(merged_df))
print("embeddings 길이:", embeddings.shape[0])

# (선택) 저장
np.save('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/merged_embeddings.npy', embeddings)
merged_df.to_csv('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/merged_chunks.csv', index=False)


merged_df 길이: 20955
embeddings 길이: 20955


In [ ]:
import torch
import numpy as np

corpus_embeddings = torch.from_numpy(
    np.load('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/merged_embeddings.npy', allow_pickle=True)
).float().to('cuda')

In [ ]:
merged_df

,chunk,source_type
0,가상자산 이용자 보호 등에 관한 법률 제1조 제1장 총칙,law
1,가상자산 이용자 보호 등에 관한 법률 제1조 이 법은 가상자산 이용자 자산의 보호와...,law
2,"가상자산 이용자 보호 등에 관한 법률 제2조 • 1. ""가상자산""이란 경제적 가치를...",law
3,가상자산 이용자 보호 등에 관한 법률 제3조 이 법은 국외에서 이루어진 행위로서 그...,law
4,가상자산 이용자 보호 등에 관한 법률 제4조 가상자산 및 가상자산사업자에 관하여 다...,law
...,...,...
20950,지그비 (Zigbee) 저속 전송속도를 갖는 홈오토메이션 및 데이터 네트워크를 위한...,ict_dictionary
20951,직교 주파수 분할 다중 (orthogonal frequency division mu...,ict_dictionary
20952,"직교 진폭 변조 (quadrature amplitude modulation), (Q...",ict_dictionary
20953,터보 코드 (Turbo code) 통신 채널의 비트 오류 확률을 최소로 유지하는 알...,ict_dictionary


In [ ]:
# from tqdm import tqdm

# # ✅ 주관식만 필터링
# subjective_df = test.loc[~test['Question'].apply(is_multiple_choice)].copy()

# preds_subjective = []

# for q in tqdm(subjective_df['Question'], desc="Inference (주관식만)"):
#     print("=" * 80)
#     print(f"[Question]\n{q}")

#     # 1) 프롬프트 생성
#     prompt = make_prompt_auto(q, merged_df, corpus_embeddings, model_embedding, top_n=3)
#     print("-" * 80)
#     print(f"[Prompt Sent to Model]\n{prompt}")

#     # 2) 모델 추론
#     output = pipe(prompt, max_new_tokens=256, temperature=0.2, top_p=0.9)
#     raw_output = output[0]["generated_text"]
#     print("-" * 80)
#     print(f"[Raw Model Output]\n{raw_output}")

#     # 3) 정답 추출
#     pred_answer = extract_answer_only(raw_output, original_question=q)
#     print("-" * 80)
#     print(f"[Extracted Final Answer] -> {pred_answer}")

#     preds_subjective.append(pred_answer)

# # 결과 저장
# subjective_df['Predicted Answer'] = preds_subjective


In [ ]:
# from tqdm import tqdm

# # ✅ 객관식만 필터링
# subjective_df = test.loc[test['Question'].apply(is_multiple_choice)].copy()

# preds_subjective = []

# for q in tqdm(subjective_df['Question'], desc="Inference (객관식만)"):
#     print("=" * 80)
#     print(f"[Question]\n{q}")

#     # 1) 프롬프트 생성
#     prompt = make_prompt_auto(q, merged_df, corpus_embeddings, model_embedding, top_n=3)
#     print("-" * 80)
#     print(f"[Prompt Sent to Model]\n{prompt}")

#     # 2) 모델 추론
#     output = pipe(prompt, max_new_tokens=64, temperature=0.2, top_p=0.9)
#     raw_output = output[0]["generated_text"]
#     print("-" * 80)
#     print(f"[Raw Model Output]\n{raw_output}")

#     # 3) 정답 추출
#     pred_answer = extract_answer_only(raw_output, original_question=q)
#     print("-" * 80)
#     print(f"[Extracted Final Answer] -> {pred_answer}")

#     preds_subjective.append(pred_answer)

# # 결과 저장
# subjective_df['Predicted Answer'] = preds_subjective


In [ ]:
# preds = []

# for q in tqdm(test['Question'], desc="Inference"):
#     prompt = make_prompt_auto(q, merged_df, corpus_embeddings, model_embedding, top_n=2)  # ✅ corpus_embeddings 사용
#     print("=" * 80)
#     print(f"Q: {q}")
#     print("\n[Prompt]\n" + prompt)
#     print("-" * 80)

#     output = pipe(prompt, max_new_tokens=128, temperature=0.2, top_p=0.9)
#     print("\n[Raw Model Output]\n", output[0]["generated_text"])

#     pred_answer = extract_answer_only(output[0]["generated_text"], original_question=q)
#     print("\n[Extracted Answer] ->", pred_answer)

#     preds.append(pred_answer)


# 파인튜닝

In [ ]:
# pip install -U transformers accelerate datasets peft

In [ ]:
# from datasets import DatasetDict, Dataset
# from transformers import TrainingArguments, Trainer
# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# import bitsandbytes as bnb
# import json

# TRAIN = "/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/파인튜닝 데이터셋/mcq_train.jsonl"
# VAL   = "/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/파인튜닝 데이터셋/mcq_val.jsonl"
# BASE  = model_name

# def load_jsonl_rows(path):
#     with open(path,"r",encoding="utf-8") as f:
#         return [json.loads(x) for x in f]

# train_rows = load_jsonl_rows(TRAIN)
# val_rows   = load_jsonl_rows(VAL)

# def build_pair(ex):
#     # 프롬프트 고정 (튜닝 시에도 “정답 번호만” 생성하도록)
#     prompt = "다음 객관식 문제의 정답 번호만 출력하라. 설명 금지.\n\n" + ex["input"] + "\n\n정답 번호:"
#     completion = " " + ex["output"]  # 앞 공백 한 칸 두면 학습 안정적
#     return {"prompt": prompt, "completion": completion}

# train_pairs = list(map(build_pair, train_rows))
# val_pairs   = list(map(build_pair, val_rows))
# ds = DatasetDict({
#     "train": Dataset.from_list(train_pairs),
#     "validation": Dataset.from_list(val_pairs)
# })

# tok = AutoTokenizer.from_pretrained(BASE, use_fast=True, trust_remote_code=True)
# tok.padding_side = "left"
# if tok.pad_token_id is None:
#     tok.pad_token = tok.eos_token or "</s>"

# MAX_LEN = 640
# IGNORE_INDEX = -100

# def tok_mask(batch):
#     p = tok(batch["prompt"], add_special_tokens=False)
#     c = tok(batch["completion"], add_special_tokens=False)
#     input_ids, attn, labels = [], [], []
#     for p_ids, c_ids in zip(p["input_ids"], c["input_ids"]):
#         max_c = MAX_LEN - len(p_ids)
#         if max_c < 1:
#             p_ids = p_ids[-(MAX_LEN-1):]
#             max_c = 1
#         if len(c_ids) > max_c:
#             c_ids = c_ids[:max_c]
#         ids = p_ids + c_ids
#         am  = [1]*len(ids)
#         if len(ids) < MAX_LEN:
#             pad = MAX_LEN - len(ids)
#             ids += [tok.pad_token_id]*pad
#             am  += [0]*pad
#         lab = [IGNORE_INDEX]*len(p_ids) + c_ids
#         if len(lab) < MAX_LEN:
#             lab += [IGNORE_INDEX]*(MAX_LEN-len(lab))
#         input_ids.append(ids); attn.append(am); labels.append(lab)
#     return {"input_ids": input_ids, "attention_mask": attn, "labels": labels}

# tok_ds = ds.map(tok_mask, batched=True, remove_columns=["prompt","completion"])

# # QLoRA (A100: 4bit + bfloat16 OK)
# model = AutoModelForCausalLM.from_pretrained(
#     BASE, load_in_4bit=True, device_map="auto",
#     torch_dtype=torch.bfloat16, trust_remote_code=True,
#     bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True
# )
# model = prepare_model_for_kbit_training(model)
# lora_cfg = LoraConfig(
#     r=16, lora_alpha=16, lora_dropout=0.05,
#     target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
# )
# model = get_peft_model(model, lora_cfg)

# from transformers import TrainingArguments

# args = TrainingArguments(
#     output_dir="/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/파인튜닝 데이터셋",
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=4,
#     num_train_epochs=1,
#     learning_rate=2e-5,
#     bf16=True,
#     logging_steps=10,
#     save_steps=100,
#     do_eval=True,
#     eval_steps=100,
#     save_total_limit=2,
#     report_to="none"
# )

# def compute_metrics(eval_pred):
#     import numpy as np
#     preds, labels = eval_pred
#     # labels에서 프롬프트(-100) 제외하고 첫 토큰만 확인
#     # (정답 한 글자이므로 첫 실제 라벨 토큰과 비교)
#     label_ids = []
#     for row in labels:
#         row = [x for x in row if x != IGNORE_INDEX]
#         label_ids.append(row[0] if row else -1)
#     # preds에서 가장 확률 높은 토큰 뽑기
#     pred_ids = preds.argmax(-1)
#     # 첫 생성 토큰만 비교
#     hits = 0; total = 0
#     for p, l in zip(pred_ids, label_ids):
#         if l == -1:
#             continue
#         total += 1
#         if p == l:
#             hits += 1
#     return {"em_token": hits/total if total else 0.0}

# trainer = Trainer(
#     model=model, args=args,
#     train_dataset=tok_ds["train"],
#     eval_dataset=tok_ds["validation"],
#     tokenizer=tok,
#     compute_metrics=compute_metrics
# )

# trainer.train()
# trainer.save_model("/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/파인튜닝 데이터셋")


In [ ]:
# from transformers import AutoTokenizer, pipeline
# from peft import AutoPeftModelForCausalLM

# ADAPTER_DIR = "/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/파인튜닝 데이터셋/checkpoint-2777"

# tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR, use_fast=True, trust_remote_code=True)
# tokenizer.padding_side = "left"
# if tokenizer.pad_token_id is None:
#     tokenizer.pad_token = tokenizer.eos_token or "</s>"

# model = AutoPeftModelForCausalLM.from_pretrained(
#     ADAPTER_DIR, device_map="auto", torch_dtype="auto", trust_remote_code=True
# ).eval()


In [ ]:
preds = []

# 두 개의 pipe 설정 (객관식 / 주관식)
pipe_mc = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    # 객관식: 길이 짧게, temperature 낮게
    max_new_tokens=64,
    temperature=0.1,
    top_p=0.9
)
pipe_short = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    # 주관식: 길이 길게, temperature 조금 높게
    max_new_tokens=256,
    temperature=0.2,
    top_p=0.9,
)

for q in tqdm(test['Question'], desc="Inference"):
    # 질문 구분
    if is_multiple_choice(q):
        used_pipe = pipe_mc
    else:
        used_pipe = pipe_short

    prompt = make_prompt_auto(q, merged_df, embeddings, model_embedding, top_n=3)
    output = used_pipe(prompt)
    pred_answer = extract_answer_only(output[0]["generated_text"], original_question=q)
    preds.append(pred_answer)


Device set to use cuda:0
Device set to use cuda:0
Inference: 100%|██████████| 515/515 [44:42<00:00,  5.21s/it]


In [ ]:
# preds = []

# for q in tqdm(test['Question'], desc="Inference"):
#     prompt = make_prompt_auto(q, merged_df, embeddings, model_embedding, top_n=3)
#     output = pipe(prompt, max_new_tokens=128, temperature=0.2, top_p=0.9)
#     pred_answer = extract_answer_only(output[0]["generated_text"], original_question=q)
#     preds.append(pred_answer)


Inference: 100%|██████████| 515/515 [1:10:52<00:00,  8.26s/it]


In [ ]:
preds

['1',
 '4',
 '2',
 '3',
 '트로이 목마 기반 원격제어 악성코드(RAT)는 정상 소프트웨어로 위장하여 사용자의 PC에 설치된 후, 공격자가 원격에서 시스템을 제어하고 정보를 탈취할 수 있도록 설계된 악성코드입니다. 주요 탐지 지표로는 평소와 달리 백그라운드에서 실행되는 불필요한 프로세스나, 정상 경로가 아닌 경로로 실행되는 프로그램, 그리고 비정상적인 네트워크 연결 시도 등이 있습니다. 또한, RAT 악성코드는 시스템 파일 변경, 이상한 파일 전송 및 수신, 원격지 서버와의 통신 로그 등 다양한 행동 패턴을 통해 탐지될 수 있습니다.',
 '4',
 '2',
 '전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관은 금융감독원의 금융분쟁조정위원회와 한국소비자원의 소비자분쟁조정위원회입니다. 이용자는 이 두 기관을 통해 전자금융거래와 관련된 분쟁을 조정받을 수 있습니다. 금융감독원과 한국소비자원은 각각 관련 법령에 따라 설치된 기관으로, 이용자의 권익 보호를 위한 분쟁조정 업무를 수행합니다.',
 '5',
 '3',
 '1',
 '4',
 '3',
 '4',
 '3',
 '4',
 '4',
 '4',
 '1',
 '4',
 '4',
 '1',
 '3',
 '5',
 '2',
 '3',
 '3',
 '3',
 '1',
 '2',
 '3',
 '3',
 '1',
 '1',
 '1',
 '2',
 '5',
 '3',
 '3',
 '4',
 '1',
 '5',
 '3',
 '4',
 '4',
 '1',
 '3',
 '1',
 '1',
 '4',
 '1',
 '3',
 '3',
 '2',
 '4',
 '4',
 '1',
 '2',
 '3',
 '2',
 '4',
 '4',
 '4',
 '5',
 '1',
 '1',
 '1',
 '1',
 '4',
 '4',
 '4',
 '4',
 '3',
 '1',
 '2',
 '2',
 '3',
 '5',
 '1',
 '4',
 '5',
 '2',
 '금융권은 딥페이크 기술 악용에 대비해 AI 기반의 진위

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/sample_submission.csv')
sample_submission['Answer'] = preds
sample_submission.to_csv('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/결과물/kanana_RAG_ver1.csv', index=False, encoding='utf-8-sig')

# 데이터 청크 LAG

In [ ]:
from sentence_transformers import SentenceTransformer
# jhgan/ko-sroberta-multitask 이 모델이 코사인 유사도 결과가 더 잘나옴
model_embedding = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [ ]:
# SentenceTransformer 사용해서 벡터 임베딩 진행
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/법령_고시_데이터_전처리_3.csv')  # 파일 경로 맞게 수정

# 법령명, 조문헤더, 본문 합치기
df['chunk'] = df['법령명'].astype(str) + " " + df['조문헤더'].astype(str) + " " + df['본문'].astype(str)

# 결과 확인
print(df[['chunk']].head())

                                               chunk
0  개인정보 보호법 시행령 제1조(목적) 이 영은 「개인정보 보호법」에서 위임된 사항과...
1  개인정보 보호법 시행령 제2조(공공기관의 범위) 「개인정보 보호법」(이하 “법”이라...
2  개인정보 보호법 시행령 제3조(영상정보처리기기의 범위) ① 법 제2조제7호에서 “대...
3  개인정보 보호법 시행령 제4조의2(영리업무의 금지) 법 제7조제1항에 따른 개인정보...
4  개인정보 보호법 시행령 제5조(전문위원회) ① 보호위원회는 법 제7조의9제1항에 따...


In [ ]:
#임베딩 모델 이미 설정 완료
chunks = df['chunk'].tolist()
embeddings = model_embedding.encode(chunks, show_progress_bar=True)

Batches:   0%|          | 0/48 [00:00<?, ?it/s]

In [ ]:
import numpy as np

np.save('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/chunk_embeddings_1.npy', embeddings)

In [ ]:
df[(df['법령명'].str.contains('개인정보 보호법')) & (df['조문헤더'].str.contains('제2조'))]

,법령명,법령타입,법령번호,시행일,조문헤더,본문,chunk
1,개인정보 보호법 시행령,대통령령,제35343호,20250701,제2조(공공기관의 범위),「개인정보 보호법」(이하 “법”이라 한다) 제2조제6호나목에서 “대통령령으로 정하는...,개인정보 보호법 시행령 제2조(공공기관의 범위) 「개인정보 보호법」(이하 “법”이라...
125,개인정보 보호법 시행령,대통령령,제35343호,20250701,제2조(결합전문기관 재지정에 관한 적용례),제29조의2제4항의 개정규정은 부칙 제1조제1항제2호에 따른 시행일 이후\n결합전문...,개인정보 보호법 시행령 제2조(결합전문기관 재지정에 관한 적용례) 제29조의2제4항...
128,개인정보 보호법,법률,제19234호,20250313,제2조(정의),"이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2014. 3. 24., 20...",개인정보 보호법 제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정...
254,개인정보 보호법,법률,제19234호,20250313,제2조(개인정보 유출 등의 통지ㆍ신고에 관한 적용례),제34조의 개정규정은 이 법 시행 이후 개인정보가 분실ㆍ도난\nㆍ유출되었음을 알게 ...,개인정보 보호법 제2조(개인정보 유출 등의 통지ㆍ신고에 관한 적용례) 제34조의 개...


In [ ]:
from sentence_transformers import util
import numpy as np

# 저장된 임베딩 불러오기
embeddings = np.load('/content/drive/MyDrive/Colab Notebooks/금융공모전_25_8/chunk_embeddings_1.npy')
chunks = df['chunk'].tolist()

# 예시 쿼리
query = "개인정보 보호법이란?"
query_emb = model_embedding.encode(query)

# 유사도 계산 (cos_scores shape: (N,))
cos_scores = util.cos_sim(query_emb, embeddings)[0].cpu().numpy() if hasattr(util.cos_sim(query_emb, embeddings)[0], "cpu") else util.cos_sim(query_emb, embeddings)[0]
top_n = 10

# 유사도 내림차순 Top N 인덱스 추출
top_idx = np.argsort(-cos_scores)[:top_n]   # <-- 여기서 바로 인덱스 생성!

print("유사도가 높은 청크:")
for rank, idx in enumerate(top_idx, 1):
    row = df.iloc[int(idx)]
    print(f"[{rank}] {row['법령명']} | {row['조문헤더']} | {row['본문'][:120]}... (score: {cos_scores[idx]:.4f})")


유사도가 높은 청크:
[1] 개인정보 보호법 | 제1조(목적) | 이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인
의 존엄과 가치를 구현함을 목적으로 한다. <개정 2014. 3. 24.>... (score: 0.7336)
[2] 개인정보 보호법 | 제2조(정의) | 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2014. 3. 24., 2020. 2. 4., 2023. 3. 14.>
1. “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 ... (score: 0.7175)
[3] 개인정보 보호법 | 제9조(개인정보파일의 등록에 관한 경과조치) | 이 법 시행 당시 종전의 제58조제1항제1호에 따른 개인정보가 포함된
개인정보파일을 운용하고 있는 공공기관의 장은 이 법 시행일부터 60일 이내에 해당 개인정보파일을 제32조의 개
법제처 42 국가법령정보센터
개인정... (score: 0.6768)
[4] 개인정보 보호법 시행령 | 제31조(개인정보 처리방침의 내용 및 공개방법 등) | ① 법 제30조제1항제8호에서 “대통령령으로 정한 사항”이란 다음
각 호의 사항을 말한다. <개정 2016. 9. 29., 2020. 8. 4., 2023. 9. 12., 2024. 3. 12.>
1. 처리하는 개인... (score: 0.6762)
[5] 개인정보 보호법 시행령 | 제33조(개인정보파일의 등록사항 등) | ① 법 제32조제1항제7호에서 “대통령령으로 정하는 사항”이란 다음 각 호의 사항
을 말한다. <개정 2023. 9. 12.>
1. 개인정보파일을 운용하는 공공기관의 명칭
2. 개인정보파일로 보유하고 있는 개인정보의... (score: 0.6754)
[6] 개인정보 보호법 | 제5조(국가 등의 책무) | ① 국가와 지방자치단체는 개인정보의 목적 외 수집, 오용ㆍ남용 및 무분별한 감시ㆍ추적 등에
따른 폐해를 방지하여 인간의 존엄과 개인의 사생활 보호를 도모하기 위한 시책을